In [85]:
import pandas as pd
import numpy as np

In [86]:
data = pd.read_csv('EV_Charging_Grid_Optimization_Categorical.csv')

In [87]:
df = pd.DataFrame(data)
df

,timestamp,station_id,location,charging_type,num_chargers,voltage_level,current_flow,power_consumed,power_loss,voltage_fluctuation,ev_id,battery_capacity,charging_time,charging_power,charging_cost,predicted_power_demand,optimized_charging_power,grid_stability_score,reduced_power_loss_category,voltage_stability_category
0,2024-01-01 00:00:00,7,Suburban,V2G,3,243.194256,10.925605,2.657044,2.152952,2.103120,4523,55.094776,37.729495,21.448731,1.991531,3.032166,3.028269,0.100286,Medium,Excellent
1,2024-01-01 00:15:00,15,Rural,Conventional,4,235.538816,74.444498,17.534569,2.344340,9.699106,1904,63.260600,52.364641,16.695066,2.151447,17.902533,18.606271,0.336108,Medium,Moderate
2,2024-01-01 00:30:00,11,Suburban,V2G,2,230.442868,73.562155,16.951874,1.357571,4.802392,1104,97.437584,68.117152,41.668615,6.985057,17.082844,16.300964,0.379212,Low,Excellent
3,2024-01-01 00:45:00,8,Suburban,V2G,5,231.152179,66.042059,15.265766,3.785360,9.484294,3047,40.414578,38.604276,24.216161,2.300621,15.238747,14.619174,0.465619,Medium,Moderate
4,2024-01-01 01:00:00,7,Urban,Fast,5,220.040609,99.109796,21.808180,2.613086,6.929790,3197,88.677620,108.880727,26.568029,7.118931,21.980294,23.483637,0.466621,Medium,Excellent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2024-01-11 08:45:00,1,Suburban,Conventional,8,246.545654,47.997301,11.833526,1.401859,8.270653,1986,75.091604,86.741911,26.608181,5.680003,11.760949,12.179340,0.940645,Low,Poor
996,2024-01-11 09:00:00,8,Suburban,V2G,9,242.172009,80.655753,19.532566,4.489497,9.334257,4118,98.292327,51.983748,10.816650,1.383773,19.964391,20.163670,0.761542,High,Moderate
997,2024-01-11 09:15:00,7,Urban,Fast,9,228.323940,27.614138,6.304969,0.607028,5.678292,1760,78.128355,144.280575,36.152886,12.836755,6.090889,5.884972,0.696231,Low,Excellent
998,2024-01-11 09:30:00,3,Urban,V2G,6,228.448481,83.822799,19.149191,3.317157,6.783929,4662,94.298350,117.870939,16.704153,4.845467,19.076349,17.813388,0.238929,Medium,Moderate


In [88]:
df.columns

Index(['timestamp', 'station_id', 'location', 'charging_type', 'num_chargers',
       'voltage_level', 'current_flow', 'power_consumed', 'power_loss',
       'voltage_fluctuation', 'ev_id', 'battery_capacity', 'charging_time',
       'charging_power', 'charging_cost', 'predicted_power_demand',
       'optimized_charging_power', 'grid_stability_score',
       'reduced_power_loss_category', 'voltage_stability_category'],
      dtype='object')

In [89]:
df['charging_type'].unique()

array(['V2G', 'Conventional', 'Fast'], dtype=object)

In [90]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [91]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Load dataset (Ensure this file exists)
df = pd.read_csv("modified_data.csv")

# Define input (X) and output (y)
input_features = [
    'num_chargers', 'charging_type', 'charging_power', 'charging_time', 
    'voltage_level', 'current_flow', 'power_consumed', 'power_loss', 
    'predicted_power_demand', 'optimized_charging_power', 'grid_stability_score', 
    'voltage_fluctuation'
]
output_feature = 'voltage_stability_category'  # Target variable

X = df[input_features].copy()  # Copy to avoid modifying the original DataFrame
y = df[output_feature]

# Encode categorical feature(s)
encoder = LabelEncoder()
X['charging_type'] = encoder.fit_transform(X['charging_type'])

# Save the encoder for inference
import joblib
joblib.dump(encoder, 'label_encoder.pkl')

# Scale numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save the scaler for inference
joblib.dump(scaler, 'scaler.pkl')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Verify shapes
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")

X_train shape: (720, 12), X_test shape: (180, 12)
y_train shape: (720,), y_test shape: (180,)


In [92]:
# Train Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate Model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.75
Classification Report:
               precision    recall  f1-score   support

   Excellent       0.78      0.70      0.74        91
        Poor       0.72      0.80      0.76        89

    accuracy                           0.75       180
   macro avg       0.75      0.75      0.75       180
weighted avg       0.75      0.75      0.75       180



In [93]:
# Pre-fit Label Encoder (do this once during preprocessing)
encoder = LabelEncoder()
df['charging_type'] = encoder.fit_transform(df['charging_type'])  # Use same encoder for training

def check_redirection(station_data):
    # Convert dictionary to DataFrame
    input_data = pd.DataFrame([station_data])

    # Apply the same label encoder for 'charging_type'
    input_data['charging_type'] = encoder.transform(input_data['charging_type'])

    # Scale the input
    input_scaled = scaler.transform(input_data)

    # Make prediction
    prediction = model.predict(input_scaled)[0]

    # Decision logic
    if prediction == "Poor":
        return "Redirect EV to the next station!"
    else:
        return "Charging is stable."

# Example test case
new_ev_data = {
    'num_chargers': 5, 'charging_type': 'Fast', 'charging_power': 50, 'charging_time': 30,
    'voltage_level': 220, 'current_flow': 40, 'power_consumed': 2000, 'power_loss': 50,
    'predicted_power_demand': 2100, 'optimized_charging_power': 1900, 'grid_stability_score': 0.7,
    'voltage_fluctuation': 0.5
}

print(check_redirection(new_ev_data))  # Should output "Charging is stable." or "Redirect EV..."

Charging is stable.


In [94]:
new_ev_data = {
    'num_chargers': 1,  # Fewer chargers may indicate lower capacity
    'charging_type': 'Fast',  # High demand may lead to instability
    'charging_power': 100,  # High power consumption
    'charging_time': 60,  # Longer charging time
    'voltage_level': 200,  # Lower voltage might indicate instability
    'current_flow': 80,  # High current can cause power fluctuations
    'power_consumed': 5000,  # High power consumption
    'power_loss': 500,  # High power loss, indicating inefficiency
    'predicted_power_demand': 6000,  # Very high demand may trigger a poor rating
    'optimized_charging_power': 4000,  # Lower than predicted demand
    'grid_stability_score': 0.3,  # Low stability score is a key factor
    'voltage_fluctuation': 2.0  # High fluctuation is a strong indicator of instability
}

print(check_redirection(new_ev_data))  # Should output "Redirect EV to the next station!"

Redirect EV to the next station!


In [95]:
new_ev_data = {
    'num_chargers': 2,  # Fewer chargers may indicate lower capacity
    'charging_type': 'Fast',  # High demand may lead to instability
    'charging_power': 100,  # High power consumption
    'charging_time': 60,  # Longer charging time
    'voltage_level': 200,  # Lower voltage might indicate instability
    'current_flow': 80,  # High current can cause power fluctuations
    'power_consumed': 10,  # High power consumption
    'power_loss': 500,  # High power loss, indicating inefficiency
    'predicted_power_demand': 6000,  # Very high demand may trigger a poor rating
    'optimized_charging_power': 4000,  # Lower than predicted demand
    'grid_stability_score': 0.3,  # Low stability score is a key factor
    'voltage_fluctuation': 2.0  # High fluctuation is a strong indicator of instability
}

print(check_redirection(new_ev_data))  # Should output "Redirect EV to the next station!"

Charging is stable.


In [96]:
import pickle

# Save the trained model
with open('grid_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [98]:
# Load the saved model
with open('grid_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [100]:
def check_redirection(station_data):
    # Convert dictionary to DataFrame
    input_data = pd.DataFrame([station_data])

    # Apply the same label encoder for 'charging_type'
    input_data['charging_type'] = encoder.transform(input_data['charging_type'])

    # Scale the input
    input_scaled = scaler.transform(input_data)

    # Make prediction
    prediction = loaded_model.predict(input_scaled)[0]

    # Decision logic
    if prediction == "Poor":
        return "Redirect EV to the next station!"
    else:
        return "Charging is stable."

In [101]:
new_ev_data = {
    'num_chargers': 2,  # Fewer chargers may indicate lower capacity
    'charging_type': 'Fast',  # High demand may lead to instability
    'charging_power': 100,  # High power consumption
    'charging_time': 60,  # Longer charging time
    'voltage_level': 200,  # Lower voltage might indicate instability
    'current_flow': 80,  # High current can cause power fluctuations
    'power_consumed': 10,  # High power consumption
    'power_loss': 500,  # High power loss, indicating inefficiency
    'predicted_power_demand': 6000,  # Very high demand may trigger a poor rating
    'optimized_charging_power': 4000,  # Lower than predicted demand
    'grid_stability_score': 0.3,  # Low stability score is a key factor
    'voltage_fluctuation': 2.0  # High fluctuation is a strong indicator of instability
}

print(check_redirection(new_ev_data))  # Should output "Redirect EV to the next station!"

Charging is stable.
